In [30]:
## this code is for the purpose of testing the functionality of the RNN

In [31]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [32]:
# Load dataset
data_file = "/workspaces/NLP_finalProject/data/completeSpamAssassin.csv"
df = pd.read_csv(data_file)
# Convert to string and handle NaNs
df["Body"] = df["Body"].fillna("").astype(str)
# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Body"])

In [33]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df["Body"], df["Label"], test_size=0.2, random_state=42
)

In [34]:
train_sequences = tokenizer.texts_to_sequences(X_train)
# train Padding
train_padded_sequences = pad_sequences(train_sequences, maxlen=200)

# # Labels
train_labels = df["Label"].values

In [35]:
## create RNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(5000, 64, input_length=200))
model.add(SimpleRNN(64))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 64)           320000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 328321 (1.25 MB)
Trainable params: 328321 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# train the model
model.fit(train_padded_sequences, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
121/121 [==============================] - 5s 31ms/step - loss: 0.6990 - accuracy: 0.5052 - val_loss: 0.6430 - val_accuracy: 0.8616
Epoch 2/10
121/121 [==============================] - 4s 30ms/step - loss: 0.6712 - accuracy: 0.6078 - val_loss: 0.6475 - val_accuracy: 0.6601
Epoch 3/10
121/121 [==============================] - 4s 33ms/step - loss: 0.6138 - accuracy: 0.6825 - val_loss: 0.8228 - val_accuracy: 0.4308
Epoch 4/10
121/121 [==============================] - 4s 29ms/step - loss: 0.5432 - accuracy: 0.7280 - val_loss: 0.6825 - val_accuracy: 0.6436
Epoch 5/10
121/121 [==============================] - 4s 29ms/step - loss: 0.4646 - accuracy: 0.7717 - val_loss: 0.7628 - val_accuracy: 0.5527
Epoch 6/10
121/121 [==============================] - 4s 31ms/step - loss: 0.4213 - accuracy: 0.7973 - val_loss: 0.9198 - val_accuracy: 0.5196
Epoch 7/10
121/121 [==============================] - 4s 30ms/step - loss: 0.3805 - accuracy: 0.8128 - val_loss: 0.8491 - val_accuracy: 0.5981

In [38]:
# test the model
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded_sequences = pad_sequences(test_sequences, maxlen=200)
test_labels = y_test.values
loss, accuracy = model.evaluate(test_padded_sequences, test_labels)
print(f"Test Accuracy: {accuracy}")

38/38 [==============================] - 0s 6ms/step - loss: 0.9465 - accuracy: 0.4942
Test Accuracy: 0.49421486258506775
